# Homework 03: Object detection

* **REMEMBER TO COPY THIS FILE TO YOUR OWN DRIVE**
* You can only modify the non maximal suppression function
* What you need to do in this homework
  * Understand YOLO and one-stage object detection models.
  * Finish the Non-maximum Suppression function.
  * Inference on sample images and finish the report.
  * [HW3 Report](https://docs.google.com/document/d/1BN-69IoHkGvBuOFyc0jxJvqsGpfmeurGrZCBHWaX_fI/edit)

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

%cd /content/gdrive/MyDrive/

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [2]:
import os
if not os.path.isdir("AI-hw3"):
    os.makedirs("AI-hw3")

%cd AI-hw3

/content/gdrive/MyDrive/AI-hw3


# Download data and checkpoint

In [3]:
# download the pretrain weight of YOLOv7
if not os.path.isfile("yolov7.pt"):
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

# download the inference data
if not os.path.isdir("data"):
  ! gdown --folder https://drive.google.com/drive/folders/1RhCVmkRD_6sF4gsgFzgUKDUrQlXJB04X?usp=share_link -O data

# Import packages

In [4]:
import torch
import random
import cv2
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import glob

from PIL import Image, ImageDraw, ImageFont

# Utility function you may need (not necessary)

In [5]:
def xyxy2xywh(x):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2  # x center
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2  # y center
    y[:, 2] = x[:, 2] - x[:, 0]  # width
    y[:, 3] = x[:, 3] - x[:, 1]  # height
    return y


def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y


def box_iou(box1, box2):
    # https://github.com/pytorch/vision/blob/master/torchvision/ops/boxes.py
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    Arguments:
        box1 (Tensor[N, 4])
        box2 (Tensor[M, 4])
    Returns:
        iou (Tensor[N, M]): the NxM matrix containing the pairwise
            IoU values for every element in boxes1 and boxes2
    """

    def box_area(box):
        # box = 4xn
        return (box[2] - box[0]) * (box[3] - box[1])

    area1 = box_area(box1.T)
    area2 = box_area(box2.T)

    # inter(N,M) = (rb(N,M,2) - lt(N,M,2)).clamp(0).prod(2)
    inter = (torch.min(box1[:, None, 2:], box2[:, 2:]) - torch.max(box1[:, None, :2], box2[:, :2])).clamp(0).prod(2)
    return inter / (area1[:, None] + area2 - inter)  # iou = inter / (area1 + area2 - inter)

# Non-Maximum Suppression(NMS)

In [6]:
def non_max_suppression(prediction, confidence_threshold=0.4, iou_thres=0.6):
    """Runs Non-Maximum Suppression (NMS) on inference results
    Returns:
         list of detections, on (n,6) tensor per image [xyxy, conf, cls]
    """

    # Start of your code

    candidates_mask = prediction[..., 4] > confidence_threshold

    outputs = list()

    for image_number, current_prediction in enumerate(prediction):
        
        current_prediction = current_prediction[candidates_mask[image_number]]  # confidence

        current_prediction[:, 5:] *= current_prediction[:, 4:5]  # conf = obj_conf * cls_conf

        boxes = xywh2xyxy(current_prediction[:, :4])

        conf, nonzero_confidence_j = current_prediction[:, 5:].max(1, keepdim=True)
          
        current_prediction = torch.cat((boxes, conf, nonzero_confidence_j.float()), 1)[conf.view(-1) > confidence_threshold]

        boxes = current_prediction[:, :4]
        scores = current_prediction[:, 4]
        sorted_idx = sorted(range(current_prediction.size()[0]), key=lambda i: scores[i], reverse=True)
 
        selected_boxes = [] # per image
        while (len(sorted_idx) > 0):
          # Pick the bounding box with the highest confidence score
          cur_idx = sorted_idx[0]
          cur_pred = current_prediction[cur_idx, :]
          cur_box = boxes[cur_idx, :]
          
          selected_boxes.append(cur_pred.tolist())

          # Calculate IoU between the current box and the remaining boxes
          remaining_boxes = [boxes[i] for i in sorted_idx[1:]]

          IoUs = [box_iou(box.unsqueeze(0), cur_box.unsqueeze(0)) for box in remaining_boxes]

          # Remove boxes with high overlap (IoU) using the specified threshold
          filtered_idx = [i for i, iou in enumerate(IoUs) if iou <= iou_thres]
          sorted_idx = [sorted_idx[i+1] for i in filtered_idx]

        outputs.append(selected_boxes)
    return outputs

    # End of your code

# Datasets

In [7]:
# build our own dataloader
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, root, transform=None):
    self.data = glob.glob(os.path.join(root,'*'))
    print(self.data)
    self.transform = transform
    self.len = len(self.data)

  def __len__(self):
    return self.len

  def __getitem__(self, index):
    image = Image.open(self.data[index])

    return transform(image), self.data[index]


yolo_img_size = 640

transform = transforms.Compose([
    transforms.Resize((yolo_img_size, yolo_img_size)),
    transforms.ToTensor()
])

# Load model and data

In [8]:
# download the yolov7 pretrain weight
model = torch.hub.load('WongKinYiu/yolov7', 'custom', 'yolov7.pt',
                        force_reload=True, trust_repo=True)


dataset = MyDataset("data", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

Downloading: "https://github.com/WongKinYiu/yolov7/zipball/main" to /root/.cache/torch/hub/main.zip


requirements: jedi>=0.16 not found and is required by YOLOR, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.2 MB/s eta 0:00:00

requirements: 1 package updated per /root/.cache/torch/hub/WongKinYiu_yolov7_main/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Adding autoShape... 
['data/1.jpg', 'data/2.jpg', 'data/3.jpg', 'data/4.jpg']


# Plot the result

In [9]:
class_labels = model.module.names if hasattr(model, 'module') else model.names

# set color for class labels
colors = [[random.randint(0, 255) for _ in range(3)] for _ in class_labels]


# draw the bounding box detect by YOLO on image
def plot_bbox(name, pred):
  img = Image.open(name)
  width, height = img.size
  for p in pred:
    label = class_labels[int(p[5])]
    box = [p[0]/yolo_img_size*width,
           p[1]/yolo_img_size*height,
           p[2]/yolo_img_size*width,
           p[3]/yolo_img_size*height]
    draw = ImageDraw.Draw(img)
    draw.rectangle(box, outline=tuple(colors[int(p[5])]), width=3)

    fontsize = max(round(max(img.size) / 40), 12)
    font = ImageFont.load_default()
    # font = ImageFont.truetype("arial.ttf", fontsize)
    txt_width, txt_height = font.getsize(label)
    draw.rectangle([box[0], box[1] - txt_height + 4, box[0] + txt_width, box[1]], fill=tuple(colors[int(p[5])]))
    draw.text((box[0], box[1] - txt_height + 1), label, fill=(255, 255, 255), font=font)
    del draw

  dor_position = name.find(".")
  save_path = name[:dor_position]+"_pred"+name[dor_position:]
  
  if not os.path.isdir("prediction"):
    os.makedirs("prediction")

  print("the result of image:", save_path, "is save to", save_path.replace("data", "prediction"))
  img.save(save_path.replace("data", "prediction"))

# Inference

In [10]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the YOLO model
model.to(device)

# Inference
model.eval()
with torch.no_grad():
  for batch_idx, (images, filename) in enumerate(dataloader):
    images = images.to(device)
    outputs = model(images)[0]
    
    # Process the outputs
    preds = non_max_suppression(outputs)

    for idx, pred in enumerate(preds):
      plot_bbox(filename[idx], pred)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


the result of image: data/3_pred.jpg is save to prediction/3_pred.jpg
the result of image: data/4_pred.jpg is save to prediction/4_pred.jpg
the result of image: data/1_pred.jpg is save to prediction/1_pred.jpg
the result of image: data/2_pred.jpg is save to prediction/2_pred.jpg
